## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import sys
sys.path.append("c:\\users\\lt\\anaconda3\\envs\\pythondata\\lib\\site-packages")
sys.path
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.80,-139.03,77.76,79,16,4.81,few clouds
1,1,Mataura,NZ,-46.19,168.86,52.48,85,100,1.87,light rain
2,2,Barrow,US,71.29,-156.79,32.02,93,100,6.69,mist
3,3,Hermanus,ZA,-34.42,19.23,50.81,85,100,2.21,overcast clouds
4,4,Soyo,AO,-6.13,12.37,74.25,88,6,3.32,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 89


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
38,38,Vega De Alatorre,MX,20.03,-96.63,86.68,64,37,6.57,scattered clouds
40,40,Hilo,US,19.73,-155.09,86.29,75,20,2.68,few clouds
55,55,Half Moon Bay,US,37.46,-122.43,87.91,70,20,2.57,few clouds
98,98,Punto Fijo,VE,11.70,-70.20,86.92,57,65,10.61,broken clouds
116,116,Umm Kaddadah,SD,13.60,26.69,88.99,7,69,4.67,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum(), preferred_cities_df.notnull().sum()

(City_ID                0
 City                   0
 Country                0
 Lat                    0
 Lng                    0
 Max Temp               0
 Humidity               0
 Cloudiness             0
 Wind Speed             0
 Current Description    0
 dtype: int64,
 City_ID                32
 City                   32
 Country                32
 Lat                    32
 Lng                    32
 Max Temp               32
 Humidity               32
 Cloudiness             32
 Wind Speed             32
 Current Description    32
 dtype: int64)

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna().copy()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
38,38,Vega De Alatorre,MX,20.03,-96.63,86.68,64,37,6.57,scattered clouds
40,40,Hilo,US,19.73,-155.09,86.29,75,20,2.68,few clouds
55,55,Half Moon Bay,US,37.46,-122.43,87.91,70,20,2.57,few clouds
98,98,Punto Fijo,VE,11.70,-70.20,86.92,57,65,10.61,broken clouds
116,116,Umm Kaddadah,SD,13.60,26.69,88.99,7,69,4.67,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
38,Vega De Alatorre,MX,86.68,scattered clouds,20.03,-96.63,
40,Hilo,US,86.29,few clouds,19.73,-155.09,
55,Half Moon Bay,US,87.91,few clouds,37.46,-122.43,
98,Punto Fijo,VE,86.92,broken clouds,11.70,-70.20,
116,Umm Kaddadah,SD,88.99,broken clouds,13.60,26.69,
128,Altamira,BR,87.89,scattered clouds,-3.20,-52.21,
140,Concord,US,88.29,clear sky,37.98,-122.03,
193,Catarama,EC,87.03,few clouds,-1.58,-79.47,
195,Pahrump,US,85.24,clear sky,36.21,-115.98,
214,Ridgecrest,US,85.68,clear sky,35.62,-117.67,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# used Module 6.5.4
# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
# hotel_df.head()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
38,Vega De Alatorre,MX,86.68,scattered clouds,20.03,-96.63,Hotel D Luxe
40,Hilo,US,86.29,few clouds,19.73,-155.09,Hilo Hawaiian Hotel
55,Half Moon Bay,US,87.91,few clouds,37.46,-122.43,"The Ritz-Carlton, Half Moon Bay"
98,Punto Fijo,VE,86.92,broken clouds,11.70,-70.20,Santa Fe Inn Hotel
128,Altamira,BR,87.89,scattered clouds,-3.20,-52.21,Requinte Hotel


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [11]:
# used Module 6.5.4
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Currrent Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))